<a href="https://colab.research.google.com/github/paulcodrea/reddit_humor/blob/main/2d_word_freq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [109]:
import csv
import nltk
from nltk import word_tokenize, RegexpTokenizer
nltk.download('stopwords')
from nltk.corpus import stopwords
stopWords = set(stopwords.words('english'))
import numpy as np
import pandas as pd
from nltk.stem.porter import *
stemmer = PorterStemmer()
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [110]:
path = 'drive/MyDrive/Humour_Detection_Dataset/final_jokes_and_facts(2918).csv'
data = pd.read_csv(path)

print("Data successfuly read!")

print("This is the first line from the data file: \n", data.head(1))


Data successfuly read!
This is the first line from the data file: 
    Unnamed: 0                                               joke  \
0           0  Did you hear about the band that had a flexibl...   

                                              tokens  token_count  funny  \
0  ['Did', 'hear', 'band', 'flexible', 'schedule'...            9      1   

                                          clean_text  
0  did hear band flexible schedule they called ru...  


In [111]:
def tokenise_clean_text(data_set):
  """
  Tokenises the data from the CSV file.
  """
  arr = []
  for line in data_set['clean_text']:
    arr.append(line.split(' '))
  arr = [x[:(len(x) - 1)] for x in arr]
  return arr

def return_vocab(document) -> list:
  """
  Retuns the vocabulary of a document.
  """
  computed_vocab = []
  for line in document:
    for word in line:
      computed_vocab.append(word)
  computed_vocab = list(dict.fromkeys(computed_vocab))
  return computed_vocab

def document_frequency(vocabulary: dict, jokes: list):
  """
  Returns a dictionary where the keys are all words in the vocabulary and the 
  values the list of jokes the word appears in.
  """
  doc_freq = {}
  for word in vocabulary:
    doc_freq[word] = 0
  for joke in jokes:
    for joke_word in joke:
      doc_freq[joke_word] = doc_freq[joke_word] + 1
  return doc_freq

def find_max_joke_size(joke_list):
  max_size = 0
  for joke in joke_list:
    if len(joke) > max_size:
      max_size = len(joke)
  return max_size

In [112]:
list_jokes_tokenized = tokenise_clean_text(data)


In [113]:
vocabulary = return_vocab(list_jokes_tokenized)

In [114]:
voc_freq = document_frequency(vocabulary, list_jokes_tokenized)

In [115]:
max_size = find_max_joke_size(list_jokes_tokenized)

In [36]:
# def jokes_as_values(vocabulary_frequency, jokes, max_size):
#   embedded_joke = np.zeros((len(jokes), max_size))
#   index_joke = 0
#   for joke in jokes:
#     index_word = 0
#     joke_list = np.zeros((len(joke)))
#     for word in joke:
#       joke_list[index_word] = vocabulary_frequency[word]
#       index_word = index_word +1
#     joke_list = sequence.pad_sequences(joke_list, maxlen=max_size)
#     embedded_joke[index_joke]= joke_list

#     index_joke = index_joke + 1
#   return embedded_joke

In [116]:
jokes_as_vectors = []
for joke in list_jokes_tokenized:
  joke_list = np.zeros((len(joke)))
  index = 0
  for word in joke:
    joke_list[index] = voc_freq[word]
    index = index + 1
  jokes_as_vectors.append(joke_list)
   


In [117]:
jokes_as_vectors = np.asarray(jokes_as_vectors)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


In [118]:
new_arr = np.zeros((len(jokes_as_vectors), max_size), dtype=int)
old_arr = jokes_as_vectors
for idx, joke in enumerate(jokes_as_vectors):
  len_joke = len(joke)
  joke_x = jokes_as_vectors[idx]
  new_arr[idx] = np.append([0] * (max_size - len_joke), joke)

print(new_arr.shape)
jokes_as_vectors = new_arr

(2918, 140)


In [119]:
# print(jokes_as_vectors[0])

In [120]:
jokes_as_vectors[:10]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,  48,  33,   7,   5,   2, 177, 105,   8,   7],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   

In [121]:
def generate_train_test_data(jokes_list, corpus, max_joke):
  train_max_index = int(len(jokes_list) * 0.8)
  X_train = np.zeros(shape=(train_max_index, max_joke), dtype=int)
  X_test = np.zeros(shape=(len(jokes_list) - train_max_index, max_joke), dtype=int)
  index_returned_data = 0
  for index in range(0, train_max_index):
    X_train[index_returned_data] = jokes_list[index]
    index_returned_data = index_returned_data + 1
  y_train = corpus["funny"][:train_max_index]
  index_returned_data = 0
  for index in range(train_max_index, len(jokes_list)):
    X_test[index_returned_data] = jokes_list[index]
    index_returned_data = index_returned_data + 1
  y_test = corpus["funny"][train_max_index:]
  return X_train, X_test, y_train, y_test

In [122]:
X_train, X_test, y_train, y_test = generate_train_test_data(jokes_as_vectors, data, max_size)

In [123]:
print(X_train[0])

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0  48  33   7   5   2 177 105   8   7]


In [124]:
print(y_train[:10])

0    1
1    1
2    0
3    0
4    1
5    1
6    0
7    0
8    0
9    0
Name: funny, dtype: int64


In [125]:
funny = 0
for item in y_train:
  funny = funny + item
print(funny)

1154


In [126]:
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(len(vocabulary), embedding_vecor_length, input_length=max_size))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 140, 32)           292288    
                                                                 
 lstm_3 (LSTM)               (None, 100)               53200     
                                                                 
 dense_3 (Dense)             (None, 1)                 101       
                                                                 
Total params: 345,589
Trainable params: 345,589
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
37/37 [==============================] - 9s 190ms/step - loss: 0.5536 - accuracy: 0.7309 - val_loss: 0.4675 - val_accuracy: 0.8014
Epoch 2/3
37/37 [==============================] - 6s 175ms/step - loss: 0.3496 - accuracy: 0.8650 - val_loss: 0.3436 - val_accuracy: 0.8545
Epoch 3/3
37/37 [========

In [102]:
import sys
np.set_printoptions(threshold=sys.maxsize)

In [127]:
# X_test
y_pred = model.predict(X_test)

In [130]:
# print(y_pred)

In [129]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 91.27%
